In [11]:
# %pip install pandas
# pip install matplotlib

In [1]:
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import random

In [6]:
## Read CSV without headers
#df = pd.read_csv("TwoLeadECG_TEST.csv", header=None)
#df = pd.read_csv("TwoLeadECG_TRAIN.csv", header=None)

## Read .txt without headers and seperated by commas
#df = pd.read_csv("Phi1.txt", header=None)  # Default is comma separator
df = pd.read_csv("Phi2.txt", header=None)  # Default is comma separator

num_columns = df.shape[1]

# Separate input features (columns 1 to end) and target (column 0)
x = torch.tensor(df.iloc[:, 1:num_columns].values, dtype=torch.float32)  # Inputs: columns 1–9
y = torch.tensor(df.iloc[:, 0].values, dtype=torch.float32).view(-1, 1)  # Target: column 0

In [ ]:
# Define a simple feedforward neural network with one hidden layer
class SimpleNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNet, self).__init__()
        self.hidden = nn.Linear(input_size, hidden_size)
        self.activation = nn.ReLU()
        #self.activation = nn.LeakyReLU()
        self.output = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.activation(self.hidden(x))
        x = self.output(x)
        return x

In [8]:
# Custom uniform norm (Chebyshev) loss function
def chebyshev_loss(output, target):
    return torch.max(torch.abs(output - target))

In [9]:
# L1 norm
loss_fn = nn.L1Loss()

In [10]:
torch.manual_seed(42)
# Example usage
if __name__ == "__main__":
    # Dummy data
    # x = torch.tensor([[0.1], [0.5], [0.9]], dtype=torch.float32)  # 3 samples, 1 feature
    # y = torch.tensor([[0.2], [0.4], [0.8]], dtype=torch.float32)  # target values

    # Network setup
    net = SimpleNet(input_size=num_columns-1, hidden_size=2, output_size=1)
    #optimizer = torch.optim.Adamax(net.parameters(), lr=0.01)
    optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

    start_time = time.time()  # Start timer

    # Training loop
    for epoch in range(100):
        optimizer.zero_grad()
        output = net(x)
        #loss = chebyshev_loss(output, y)
        loss = loss_fn(output, y)
        loss.backward()
        optimizer.step()

        #if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.6f}")

    end_time = time.time()  # End timer

elapsed_time = end_time - start_time
print(f"\n Total training time: {elapsed_time:.4f} seconds")

Epoch 0, Loss: 1.361376
Epoch 1, Loss: 1.342299
Epoch 2, Loss: 1.323335
Epoch 3, Loss: 1.304441
Epoch 4, Loss: 1.285636
Epoch 5, Loss: 1.266945
Epoch 6, Loss: 1.248409
Epoch 7, Loss: 1.230033
Epoch 8, Loss: 1.211840
Epoch 9, Loss: 1.193788
Epoch 10, Loss: 1.175865
Epoch 11, Loss: 1.158038
Epoch 12, Loss: 1.140391
Epoch 13, Loss: 1.122938
Epoch 14, Loss: 1.105744
Epoch 15, Loss: 1.088693
Epoch 16, Loss: 1.071814
Epoch 17, Loss: 1.055245
Epoch 18, Loss: 1.039022
Epoch 19, Loss: 1.023261
Epoch 20, Loss: 1.007925
Epoch 21, Loss: 0.993077
Epoch 22, Loss: 0.978675
Epoch 23, Loss: 0.964744
Epoch 24, Loss: 0.951218
Epoch 25, Loss: 0.938069
Epoch 26, Loss: 0.925459
Epoch 27, Loss: 0.913180
Epoch 28, Loss: 0.901314
Epoch 29, Loss: 0.889810
Epoch 30, Loss: 0.878820
Epoch 31, Loss: 0.868223
Epoch 32, Loss: 0.858028
Epoch 33, Loss: 0.848305
Epoch 34, Loss: 0.839078
Epoch 35, Loss: 0.830224
Epoch 36, Loss: 0.821711
Epoch 37, Loss: 0.813631
Epoch 38, Loss: 0.806062
Epoch 39, Loss: 0.798911
Epoch 40, 

In [14]:
# ============================================================
# ADD THE SIGN-DISTRIBUTION CODE HERE (FINAL STEP)
# ============================================================

with torch.no_grad():
    z = net.hidden(x)        # shape (N, hidden_size)
    signs = torch.sign(z)    # -1 or +1 (0 extremely unlikely)

print("Signs at last epoch (per sample, per hidden node):")
print(signs)

Signs at last epoch (per sample, per hidden node):
tensor([[-1., -1.],
        [-1., -1.],
        [-1., -1.],
        ...,
        [ 1.,  1.],
        [ 1.,  1.],
        [ 1.,  1.]])
